In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
!pip install geopandas

In [ ]:
!pip install typing_extensions==4.5.0 --upgrade
!pip install openai
!pip install tiktoken
!pip install --upgrade transformers
!pip install langchain
!pip install -U langchain-openai
!pip install torch

  Attempting uninstall: typing_extensions
    Found existing installation: typing_extensions 4.9.0
    Uninstalling typing_extensions-4.9.0:
      Successfully uninstalled typing_extensions-4.9.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
fastapi 0.109.0 requires typing-extensions>=4.8.0, but you have typing-extensions 4.5.0 which is incompatible.
openai 1.10.0 requires typing-extensions<5,>=4.7, but you have typing-extensions 4.5.0 which is incompatible.
  Using cached typing_extensions-4.9.0-py3-none-any.whl (32 kB)
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0
ERROR: pip's dependency resolve

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
import shapely

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
path = '/content/drive/MyDrive/dataset/madison/'

## Load dataset

In [ ]:
objects = pd.read_csv(path + '/input/objects_embeddings.csv')
objects[['embedding_ada']] = objects[['embedding_ada']].applymap(lambda s: np.array(eval(s)))
objects[['embedding_large']] = objects[['embedding_large']].applymap(lambda s: np.array(eval(s)))
objects.head()

,objectid,geometry,source,geom_type,n_tokens,embedding_ada,embedding_large
0,"road_(6231530142, 3861040918)","LINESTRING (-89.4719 43.05, -89.472 43.05)",link,LineString,21,"[-0.014463399536907673, -0.004633771721273661,...","[0.01921754889190197, 0.01929466612637043, -0...."
1,"road_(53591318, 1179412798)","LINESTRING (-89.477 43.0693, -89.477 43.0681)",link,LineString,22,"[-0.014376881532371044, -0.004993679467588663,...","[0.021175693720579147, 0.015708766877651215, -..."
2,"road_(1178722373, 1178722211)","LINESTRING (-89.4626 43.0553, -89.4627 43.0554)",link,LineString,24,"[-0.015243860892951488, -0.004274553153663874,...","[0.011615517549216747, 0.02089725434780121, -0..."
3,"road_(53430628, 53468282)","LINESTRING (-89.3045 43.0246, -89.3053 43.0252)",link,LineString,24,"[-0.016633406281471252, -0.0014020365197211504...","[0.0076748463325202465, 0.030930323526263237, ..."
4,"road_(3444117858, 6680557104)","LINESTRING (-89.5081 43.0066, -89.5072 43.006)",link,LineString,23,"[-0.010978729464113712, -0.005506434012204409,...","[0.03258766606450081, 0.024168409407138824, -0..."


In [ ]:

relations = pd.read_csv(path+'input/relations.csv', usecols=['subjectid', 'objectid', 'predicate', 'relate', 'idx'])
# relations.to_csv('/content/drive/MyDrive/dataset/madison/output/relations_update.csv', index=False)
# relations = pd.read_csv('/content/drive/MyDrive/dataset/madison/output/relations_update.csv')
relations.head()

,objectid,subjectid,predicate,relate,idx
0,"road_(53496379, 5657142535)","road_(5656899646, 53496379)",contains,101F00FF2,0
1,"road_(2634716954, 8997363243)","road_(8997363242, 2634716954)",contains,101F00FF2,1
2,poi_way/1134399612,poi_way/1134399613,contains,1F1FFFFF2,2
3,"road_(1998162788, 1998162839)",poi_way/257174438,contains,101FFFFF2,3
4,"road_(3420767254, 3590603510)",poi_way/287637061,contains,101FFFFF2,4


In [ ]:
relations = pd.merge(relations, objects[['objectid', 'geometry', 'source', 'geom_type','n_tokens']].rename(columns={'objectid': 'subjectid',
                                                        'geometry': 'geometry_subject',
                                                        'source': 'source_subject',
                                                        'geom_type': 'geom_type_subject',
                                                          'n_tokens': 'n_tokens_subject'
                                                        }), on='subjectid', how='inner')
relations = pd.merge(relations, objects[['objectid', 'geometry', 'source', 'geom_type', 'n_tokens']], on='objectid', how='inner')
relations.head()

,objectid,subjectid,predicate,relate,idx,geometry_subject,source_subject,geom_type_subject,n_tokens_subject,geometry,source,geom_type,n_tokens
0,"road_(53496379, 5657142535)","road_(5656899646, 53496379)",contains,101F00FF2,0,"LINESTRING (-89.3397 43.0922, -89.3386 43.0922)",link,LineString,24,"LINESTRING (-89.3386 43.0922, -89.3387 43.0922)",link,LineString,24
1,"road_(53496379, 5657142535)","road_(53496379, 5656899646)",contains,101F00FF2,31,"LINESTRING (-89.3386 43.0922, -89.3397 43.0922)",link,LineString,24,"LINESTRING (-89.3386 43.0922, -89.3387 43.0922)",link,LineString,24
2,"road_(5657142535, 53496379)","road_(5656899646, 53496379)",contains,101F00FF2,202,"LINESTRING (-89.3397 43.0922, -89.3386 43.0922)",link,LineString,24,"LINESTRING (-89.3387 43.0922, -89.3386 43.0922)",link,LineString,24
3,"road_(5657142535, 53496379)","road_(53496379, 5656899646)",contains,101F00FF2,54,"LINESTRING (-89.3386 43.0922, -89.3397 43.0922)",link,LineString,24,"LINESTRING (-89.3387 43.0922, -89.3386 43.0922)",link,LineString,24
4,"road_(5657142535, 53496379)",census_550250020003,contains,102FF1FF2,180,"POLYGON ((-89.3515 43.0928, -89.346 43.0939, -...",cbg,Polygon,109,"LINESTRING (-89.3387 43.0922, -89.3386 43.0922)",link,LineString,24


In [ ]:
relations[(relations['geom_type_subject']=='Point')&(relations['geom_type']=='LineString')].sort_values(by=['n_tokens_subject', 'n_tokens']).iloc[0]

objectid                            road_(2633805462, 3423901632)
subjectid                                         node_3752320724
predicate                                                disjoint
relate                                                  FF0FFF102
idx                                                           195
geometry_subject                           POINT (-89.456 43.061)
source_subject                                       intersection
geom_type_subject                                           Point
n_tokens_subject                                               10
geometry             LINESTRING (-89.459 43.0661, -89.459 43.066)
source                                                       link
geom_type                                              LineString
n_tokens                                                       21
Name: 1383, dtype: object

In [ ]:
relate_df = relations[['geom_type_subject', 'predicate', 'geom_type']].drop_duplicates().reset_index(drop=True)
relate_df['rid'] = relate_df.index
relate_df['output'] = '(' + relate_df['geom_type_subject'] + ',' + relate_df['predicate'] + ',' + relate_df['geom_type'] + ')'
relations = pd.merge(relations, relate_df, on=['geom_type_subject', 'predicate', 'geom_type'], how='left')
# relations = pd.merge(relations, relate_df, on=['geom_type_subject', 'predicate', 'geom_type'], how='left')

In [ ]:
relations[['objectid', 'geom_type', 'geom_type_subject', 'predicate']].groupby(['geom_type_subject', 'predicate', 'geom_type']).count()

objectid
geom_type_subject predicate geom_type           
LineString        contains  LineString       225
                            Point            225
                  crosses   LineString       225
                            Polygon          225
                  disjoint  LineString       225
                            Point            225
                            Polygon          225
                  equals    LineString       225
                  overlaps  LineString       225
                  touches   LineString       225
                            Point            225
                            Polygon          225
                  within    LineString       225
                            Polygon          225
Point             disjoint  LineString       225
                            Point            225
                            Polygon          225
                  equals    Point            225
                  touches   LineString       225
                            Polygon          225
                  within    LineString       225
                            Polygon          225
Polygon           contains  LineString       225
                            Point            225
                            Polygon          225
                  crosses   LineString       225
                  disjoint  LineString       225
                            Point            225
                            Polygon          225
                  equals    Polygon          225
                  overlaps  Polygon          225
                  touches   LineString       225
                            Point            225
                            Polygon          225
                  within    Polygon          225

In [ ]:
relations[['objectid', 'predicate']].groupby(['predicate']).count()

,objectid
predicate,
contains,1125
crosses,675
disjoint,2025
equals,675
overlaps,450
touches,1800
within,1125


In [ ]:
intersections = pd.read_csv(path + 'input/intersections.csv')
intersections.head()

,objectid,geometry,subjectid,geometry_subject,geom_type_subject,geom_type,predicate,pid,relate
0,"road_(6231530142, 3861040918)","LINESTRING (-89.4719 43.05, -89.472 43.05)","road_(6231530142, 3861040918)","LINESTRING (-89.4719 43.05, -89.472 43.05)",LineString,LineString,equals,0,1FFF0FFF2
1,node_3455066029,POINT (-89.4513 43.0502),node_3455066029,POINT (-89.4513 43.0502),Point,Point,equals,0,0FFFFFFF2
2,node_3455066034,POINT (-89.451 43.0502),node_3455066034,POINT (-89.451 43.0502),Point,Point,equals,0,0FFFFFFF2
3,"road_(53509392, 53412067)","LINESTRING (-89.451 43.0476, -89.451 43.0484)","road_(53509392, 53412067)","LINESTRING (-89.451 43.0476, -89.451 43.0484)",LineString,LineString,equals,0,1FFF0FFF2
4,node_1142903570,POINT (-89.3634 43.0935),node_1142903570,POINT (-89.3634 43.0935),Point,Point,equals,0,0FFFFFFF2


## Train / Test split

In [ ]:
relations_example = relations[relations['idx']>=200].reset_index(drop=True)

relations_test = relations[relations['idx']<40].reset_index(drop=True)

relations_train = relations[(relations['idx']>=40)&(relations['idx']<200)].reset_index(drop=True)

In [ ]:
relations.head()

,objectid,subjectid,predicate,relate,idx,geometry_subject,source_subject,geom_type_subject,n_tokens_subject,geometry,source,geom_type,n_tokens,rid,output
0,"road_(53496379, 5657142535)","road_(5656899646, 53496379)",contains,101F00FF2,0,"LINESTRING (-89.3397 43.0922, -89.3386 43.0922)",link,LineString,24,"LINESTRING (-89.3386 43.0922, -89.3387 43.0922)",link,LineString,24,0,"(LineString,contains,LineString)"
1,"road_(53496379, 5657142535)","road_(53496379, 5656899646)",contains,101F00FF2,31,"LINESTRING (-89.3386 43.0922, -89.3397 43.0922)",link,LineString,24,"LINESTRING (-89.3386 43.0922, -89.3387 43.0922)",link,LineString,24,0,"(LineString,contains,LineString)"
2,"road_(5657142535, 53496379)","road_(5656899646, 53496379)",contains,101F00FF2,202,"LINESTRING (-89.3397 43.0922, -89.3386 43.0922)",link,LineString,24,"LINESTRING (-89.3387 43.0922, -89.3386 43.0922)",link,LineString,24,0,"(LineString,contains,LineString)"
3,"road_(5657142535, 53496379)","road_(53496379, 5656899646)",contains,101F00FF2,54,"LINESTRING (-89.3386 43.0922, -89.3397 43.0922)",link,LineString,24,"LINESTRING (-89.3387 43.0922, -89.3386 43.0922)",link,LineString,24,0,"(LineString,contains,LineString)"
4,"road_(5657142535, 53496379)",census_550250020003,contains,102FF1FF2,180,"POLYGON ((-89.3515 43.0928, -89.346 43.0939, -...",cbg,Polygon,109,"LINESTRING (-89.3387 43.0922, -89.3386 43.0922)",link,LineString,24,1,"(Polygon,contains,LineString)"


In [ ]:
relations_test.sort_values(by=['rid', 'idx']).head()

,objectid,subjectid,predicate,relate,idx,geometry_subject,source_subject,geom_type_subject,n_tokens_subject,geometry,source,geom_type,n_tokens,rid,output
0,"road_(53496379, 5657142535)","road_(5656899646, 53496379)",contains,101F00FF2,0,"LINESTRING (-89.3397 43.0922, -89.3386 43.0922)",link,LineString,24,"LINESTRING (-89.3386 43.0922, -89.3387 43.0922)",link,LineString,24,0,"(LineString,contains,LineString)"
2,"road_(2634716954, 8997363243)","road_(8997363242, 2634716954)",contains,101F00FF2,1,"LINESTRING (-89.3554 43.1351, -89.3554 43.1357)",link,LineString,24,"LINESTRING (-89.3554 43.1357, -89.3554 43.1355)",link,LineString,24,0,"(LineString,contains,LineString)"
3,poi_way/1134399612,poi_way/1134399613,contains,1F1FFFFF2,2,"LINESTRING (-89.4524 43.0716, -89.4521 43.0716...",poi,LineString,57,"LINESTRING (-89.4524 43.0715, -89.4521 43.0715...",poi,LineString,46,0,"(LineString,contains,LineString)"
6,"road_(1998162788, 1998162839)",poi_way/257174438,contains,101FFFFF2,3,"LINESTRING (-89.3392 43.0637, -89.3399 43.0641...",poi,LineString,134,"LINESTRING (-89.338 43.0659, -89.3378 43.0659)",link,LineString,23,0,"(LineString,contains,LineString)"
10,"road_(3420767254, 3590603510)",poi_way/287637061,contains,101FFFFF2,4,"LINESTRING (-89.5168 43.0606, -89.5168 43.0616...",poi,LineString,57,"LINESTRING (-89.5175 43.0611, -89.5175 43.0607)",link,LineString,24,0,"(LineString,contains,LineString)"


In [ ]:
relations_test[['objectid', 'geom_type', 'geom_type_subject', 'predicate']].groupby(['geom_type_subject', 'predicate', 'geom_type']).count()

objectid
geom_type_subject predicate geom_type           
LineString        contains  LineString        40
                            Point             40
                  crosses   LineString        40
                            Polygon           40
                  disjoint  LineString        40
                            Point             40
                            Polygon           40
                  equals    LineString        40
                  overlaps  LineString        40
                  touches   LineString        40
                            Point             40
                            Polygon           40
                  within    LineString        40
                            Polygon           40
Point             disjoint  LineString        40
                            Point             40
                            Polygon           40
                  equals    Point             40
                  touches   LineString        40
                            Polygon           40
                  within    LineString        40
                            Polygon           40
Polygon           contains  LineString        40
                            Point             40
                            Polygon           40
                  crosses   LineString        40
                  disjoint  LineString        40
                            Point             40
                            Polygon           40
                  equals    Polygon           40
                  overlaps  Polygon           40
                  touches   LineString        40
                            Point             40
                            Polygon           40
                  within    Polygon           40

In [ ]:
data = relations_test[relations_test['predicate']!='disjoint'].reset_index(drop=True)

## Authentication

In [ ]:
# import tiktoken, os, openai

from google.colab import userdata

KEY = userdata.get('OPENAI_QSR')
ORG = userdata.get('OPENAI_ORG')
# openai.organization = ORG
# openai.api_key = KEY
# openai.Model.list()

In [ ]:
model = 'gpt-4'
llm_abbr = 'gpt4'
model_abbr = 'gpt4'

In [ ]:
from langchain_openai import ChatOpenAI
# process.env.OPENAI_ORGANIZATION = ORG


In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

### Create vectordb

In [ ]:
!pip install chromadb

In [ ]:
embedding_model = "text-embedding-3-large" #"text-embedding-ada-002" #"text-embedding-3-large" #"text-embedding-ada-002" #"text-embedding-3-large" #"text-embedding-ada-002"
embed_abbr = "large" #"ada" #"large" #"ada"

In [ ]:
objects['vector_id'] = np.arange(objects.shape[0]).astype(str)
objects['embedding_%s'%embed_abbr] = objects['embedding_%s'%embed_abbr].apply(lambda s: s.tolist())

In [ ]:
# create collections
from langchain.embeddings.openai import OpenAIEmbeddings
# from chromadb.utils.embedding_functions import OpenAIEmbeddingFunction
import chromadb

chroma_client = chromadb.EphemeralClient()

embedding_function = OpenAIEmbeddings(api_key=KEY, organization=ORG, model=embedding_model)

try:
    chroma_client.get_collection('objects')
    chroma_client.delete_collection('objects')
except:
    pass
objects_collection = chroma_client.create_collection(name='objects',
                                                     metadata={"hnsw:space": "cosine"})

# Add the content vectors
objects_collection.add(
    ids=objects.vector_id.tolist(),
    embeddings=objects['embedding_%s'%embed_abbr].tolist(),
    documents=objects.geometry.tolist(),
    metadatas=objects[['objectid']].to_dict('records')
)

In [ ]:
from langchain_community.vectorstores import Chroma

db = Chroma(
    client=chroma_client,
    collection_name="objects",
    embedding_function=embedding_function,
)


In [ ]:
data = relations_test[relations_test['predicate']!='disjoint']

In [ ]:
include_type='t'
entity = 'subject'

,objectid,subjectid,predicate,relate,idx,geometry_subject,source_subject,geom_type_subject,n_tokens_subject,geometry,source,geom_type,n_tokens,rid,output,rev


In [ ]:
predicate_df

,predicate,rev
0,equals,equals
1,within,is contained by
2,contains,is within
3,crosses,is crossed by
4,overlaps,overlaps
5,touches,touches
6,disjoint,is disjoint to


## Direct query - orginal

In [ ]:
import time
# llm_abbr = 'none'
qe_mode = 'none'
dir = 'orig'

k_val = [5, 10, 20]

ranks = []
for idx, row in data.iloc[:].iterrows():
    text = row['geometry_subject'] + ' ' + row['predicate']
    text = row['geometry_subject'] + ' is ' + row['predicate'] if row['predicate']=='within' else text
    question = 'Retrieve a ' + row['geom_type'] + ' which ' + text


    subjects = db.similarity_search_with_score(question, k=objects.shape[0])
    dists = []
    for subject in subjects:
        dists.append({'objectid': subject[0].metadata['objectid'], 'dist': subject[1]})
    dists = pd.DataFrame(dists)
    positive = set(intersections['objectid'][(intersections['subjectid']==row['subjectid'])&
                                            (intersections['predicate']==row['predicate'])&
                                             (intersections['geom_type']==row['geom_type'])]) - set([row['objectid']])

    dists = dists[~(dists['objectid'].isin(positive))].sort_values(by='dist')
    dists['rank'] = np.arange(dists.shape[0])+1
    rank = dists[dists['objectid']==row['objectid']].iloc[0]
    ranks.append({'idx': idx, 'rank': rank['rank'], 'dist': rank['dist'],
                  'list': dists['objectid'].iloc[:10].values.tolist()})

    if (idx+1)%50 == 0:
        res_search = pd.DataFrame(ranks)
        mrr = (1./res_search['rank'].values).mean()
        hits = []
        for k in k_val:
            hits.append(np.array(res_search['rank'].values<=k).mean())

        res_search.to_csv(path+'result/temp/task2_object_%s_%s.csv'%(qe_mode, dir), index=False)
        print('----------------')
        print('record %d'%(idx+1), 'mrr: %s, hits: %s'%(mrr, hits))
        print('finish saving first %d searching with %s QE, using %s as embedding model, %s as llm.with geom_type, % s'%(
            idx+1, qe_mode, embed_abbr, llm_abbr, include_type))
        time.sleep(5)

----------------
record 50 mrr: 0.02019352292525096, hits: [0.04, 0.06, 0.06]
finish saving first 50 searching with none QE, using large as embedding model, gpt4 as llm.with geom_type, t


----------------
record 100 mrr: 0.010786767188268188, hits: [0.02, 0.03, 0.03]
finish saving first 100 searching with none QE, using large as embedding model, gpt4 as llm.with geom_type, t


----------------
record 150 mrr: 0.014458546482030361, hits: [0.02, 0.02666666666666667, 0.02666666666666667]
finish saving first 150 searching with none QE, using large as embedding model, gpt4 as llm.with geom_type, t


----------------
record 200 mrr: 0.014050802844188439, hits: [0.02, 0.03, 0.035]
finish saving first 200 searching with none QE, using large as embedding model, gpt4 as llm.with geom_type, t


----------------
record 250 mrr: 0.02106643098562671, hits: [0.032, 0.052, 0.06]
finish saving first 250 searching with none QE, using large as embedding model, gpt4 as llm.with geom_type, t


...

----------------
record 800 mrr: 0.04854119715596058, hits: [0.065, 0.0975, 0.1325]
finish saving first 800 searching with none QE, using large as embedding model, gpt4 as llm.with geom_type, t


----------------
record 850 mrr: 0.05212897840948338, hits: [0.06823529411764706, 0.1011764705882353, 0.14]
finish saving first 850 searching with none QE, using large as embedding model, gpt4 as llm.with geom_type, t


----------------
record 900 mrr: 0.05021939434000872, hits: [0.06666666666666667, 0.1, 0.13777777777777778]
finish saving first 900 searching with none QE, using large as embedding model, gpt4 as llm.with geom_type, t


----------------
record 950 mrr: 0.06454493293690093, hits: [0.08105263157894736, 0.11473684210526315, 0.15368421052631578]
finish saving first 950 searching with none QE, using large as embedding model, gpt4 as llm.with geom_type, t


----------------
record 1000 mrr: 0.08818932766351517, hits: [0.11, 0.147, 0.188]
finish saving first 1000 searching with none QE, using large as embedding model, gpt4 as llm.with geom_type, t


In [ ]:
res = data[['subjectid', 'objectid', 'predicate']].reset_index().copy()
res_search = pd.DataFrame(ranks)
res = pd.concat([res, res_search], axis=1)


res.to_csv(path+'result/task2/task2_object_%s_%s.csv'%(qe_mode, dir), index=False)
res.head()

,index,subjectid,objectid,predicate,idx,rank,dist,list
0,0,"road_(5656899646, 53496379)","road_(53496379, 5657142535)",contains,0,139,0.247416,"[road_(5656899646, 53496379), road_(53496379, ..."
1,1,"road_(53496379, 5656899646)","road_(53496379, 5657142535)",contains,1,21,0.236763,"[road_(53496379, 5656899646), line_road_(41724..."
2,2,"road_(8997363242, 2634716954)","road_(2634716954, 8997363243)",contains,2,2,0.239055,"[road_(8997363242, 2634716954), road_(26347169..."
3,3,poi_way/1134399613,poi_way/1134399612,contains,3,254,0.288741,"[poi_way/1134399581, poi_way/1134399566, poi_w..."
4,4,parcel_70920121012,poi_way/1134399612,contains,4,1341,0.293429,"[census_550250023012, polygon_census_550250103..."


In [ ]:
res_search.to_csv(path+'result/task2/task2_object_%s_%s.csv'%(qe_mode, dir), index=False)

In [ ]:
k_val = [5, 10, 20]

mrr = (1./res['rank']).mean()
hits = []
for k in k_val:
    hits.append(np.array(res['rank']<=k).mean())

mrr, hits

(0.10544103552206227,
 [0.13076923076923078, 0.1701923076923077, 0.21057692307692308])

## Direct Query - reverse

In [ ]:
predicate_df = pd.DataFrame({'predicate': ['equals', 'within', 'contains', 'crosses', 'overlaps', 'touches', 'disjoint'],
                          'rev': ['equals', 'is contained by', 'is within', 'is crossed by', 'overlaps', 'touches', 'is disjoint to']})
data = pd.merge(data, predicate_df, on='predicate', how='inner')
data[pd.isnull(data['rev'])]

In [ ]:
import time
# llm_abbr = 'none'
qe_mode = 'none'
dir = 'rev'

k_val = [5, 10, 20]

ranks = []
for idx, row in data.iloc[:].iterrows():
    text = row['rev'] + ' ' + row['geometry_subject']
    # text =  ' is ' + text if row['rev']=='within' else text
    question = 'Retrieve a ' + row['geom_type'] +  ' which ' + text


    subjects = db.similarity_search_with_score(question, k=objects.shape[0])
    dists = []
    for subject in subjects:
        dists.append({'objectid': subject[0].metadata['objectid'], 'dist': subject[1]})
    dists = pd.DataFrame(dists)
    positive = set(intersections['objectid'][(intersections['subjectid']==row['subjectid'])&
                                            (intersections['predicate']==row['predicate'])&
                                              (intersections['geom_type']==row['geom_type'])]) - set([row['objectid']])

    dists = dists[~(dists['objectid'].isin(positive))].sort_values(by='dist')
    dists['rank'] = np.arange(dists.shape[0])+1
    rank = dists[dists['objectid']==row['objectid']].iloc[0]
    ranks.append({'idx': idx, 'rank': rank['rank'], 'dist': rank['dist'],
                  'list': dists['objectid'].iloc[:10].values.tolist()})

    if (idx+1)%50 == 0:
        res_search = pd.DataFrame(ranks)
        mrr = (1./res_search['rank'].values).mean()
        hits = []
        for k in k_val:
            hits.append(np.array(res_search['rank'].values<=k).mean())

        res_search.to_csv(path+'result/temp/task2_object_%s_%s.csv'%(qe_mode, dir), index=False)
        print('----------------')
        print('record %d'%(idx+1), 'mrr: %s, hits: %s'%(mrr, hits))
        print('finish saving first %d searching with %s QE, using %s as embedding model, %s as llm.with geom_type, % s'%(
            idx+1, qe_mode, embed_abbr, llm_abbr, include_type))
        time.sleep(5)

----------------
record 50 mrr: 0.10721725237842808, hits: [0.24, 0.34, 0.38]
finish saving first 50 searching with none QE, using large as embedding model, gpt4 as llm.with geom_type, t


----------------
record 100 mrr: 0.054653643580899934, hits: [0.12, 0.17, 0.2]
finish saving first 100 searching with none QE, using large as embedding model, gpt4 as llm.with geom_type, t


----------------
record 150 mrr: 0.042356997805676366, hits: [0.09333333333333334, 0.12666666666666668, 0.15333333333333332]
finish saving first 150 searching with none QE, using large as embedding model, gpt4 as llm.with geom_type, t


----------------
record 200 mrr: 0.04224806626698063, hits: [0.09, 0.115, 0.135]
finish saving first 200 searching with none QE, using large as embedding model, gpt4 as llm.with geom_type, t


----------------
record 250 mrr: 0.05405392580119647, hits: [0.108, 0.144, 0.176]
finish saving first 250 searching with none QE, using large as embedding model, gpt4 as llm.with geom_type, t


...

----------------
record 800 mrr: 0.0745411832163236, hits: [0.13125, 0.16875, 0.205]
finish saving first 800 searching with none QE, using large as embedding model, gpt4 as llm.with geom_type, t


----------------
record 850 mrr: 0.0777221579275729, hits: [0.1376470588235294, 0.1776470588235294, 0.2188235294117647]
finish saving first 850 searching with none QE, using large as embedding model, gpt4 as llm.with geom_type, t


----------------
record 900 mrr: 0.07571012154999672, hits: [0.13555555555555557, 0.17333333333333334, 0.21222222222222223]
finish saving first 900 searching with none QE, using large as embedding model, gpt4 as llm.with geom_type, t


----------------
record 950 mrr: 0.09414667417539448, hits: [0.15578947368421053, 0.1936842105263158, 0.2326315789473684]
finish saving first 950 searching with none QE, using large as embedding model, gpt4 as llm.with geom_type, t


----------------
record 1000 mrr: 0.12446394364122793, hits: [0.192, 0.23, 0.27]
finish saving first 1000 searching with none QE, using large as embedding model, gpt4 as llm.with geom_type, t


In [ ]:
question

'Retrieve a Polygon which equals POLYGON ((-89.2871 43.1138, -89.2878 43.1136, -89.2877 43.1139, -89.2871 43.1139, -89.2871 43.11382, -89.2871 43.1138))'

In [ ]:
row

objectid                                           parcel_081034401026
subjectid                                  polygon_parcel_081034401026
predicate                                                       equals
relate                                                       2FFF1FFF2
idx                                                                 39
geometry_subject     POLYGON ((-89.2871 43.1138, -89.2878 43.1136, ...
source_subject                                          equals_polygon
geom_type_subject                                              Polygon
n_tokens_subject                                                    69
geometry             POLYGON ((-89.2878 43.1136, -89.2877 43.1139, ...
source                                                          parcel
geom_type                                                      Polygon
n_tokens                                                            58
rid                                                                 32
output

In [ ]:
res = data[['subjectid', 'objectid', 'predicate']].reset_index().copy()
res_search = pd.DataFrame(ranks)
res = pd.concat([res, res_search], axis=1)


res.to_csv(path+'result/task2/task2_object_%s_%s.csv'%(qe_mode, dir), index=False)
res.head()

,index,subjectid,objectid,predicate,idx,rank,dist,list
0,0,"road_(5656899646, 53496379)","road_(53496379, 5657142535)",contains,0,8,0.226426,"[road_(5656899646, 53496379), road_(53496379, ..."
1,1,"road_(53496379, 5656899646)","road_(53496379, 5657142535)",contains,1,2,0.217015,"[road_(53496379, 5656899646), road_(53496379, ..."
2,2,"road_(8997363242, 2634716954)","road_(2634716954, 8997363243)",contains,2,2,0.231858,"[road_(8997363242, 2634716954), road_(26347169..."
3,3,poi_way/1134399613,poi_way/1134399612,contains,3,16,0.224386,"[poi_way/1134399613, poi_way/107454783, poi_wa..."
4,4,parcel_70920121012,poi_way/1134399612,contains,4,2016,0.298383,"[parcel_70920121012, census_550250015023, cens..."


In [ ]:
k_val = [5, 10, 20]

mrr = (1./res['rank']).mean()
hits = []
for k in k_val:
    hits.append(np.array(res['rank']<=k).mean())

mrr, hits

(0.1515224053221487,
 [0.21923076923076923, 0.25576923076923075, 0.2971153846153846])

## Zero-shot-original

#### Generate pseudo geometry

In [ ]:
llm = ChatOpenAI(model_name=model, temperature=0.7, api_key=KEY, organization=ORG)

In [ ]:
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.chains import LLMChain
import langchain
import time

# llm_abbr = 'gpt3_5'

# include_type = 'f'
k_val = [5, 10, 20]

qe_mode = 'q2d_zs'
dir = 'orig'

SYSTEM_PROMPT = """You will receive a spatial query describing the subject A (WKT(A) and a spatial relationship PREDICATE, and
the desired geometry type of the object to be retrieved.
Your task is to generate a plausible geometry B in WKT format that could be the result of the query.
The PREDICATE is defined according to the DE-9IM and Open Geospatial Consortium standards.

- Provide only the WKT(B) as the output, and nothing else.
- MAKE SURE the WKT(B) represents a valid Point, LineString, or Polygon as required by the query.
- MAKE SURE the PREDICATE satisfies the the dimension requirements defined by DE-9IM and OGC given Geometry Type A and Geometry Type B.
"""

USER_PROMPT = """
{query}.
"""

PROMPT = ChatPromptTemplate.from_messages(
    [
        ("system", SYSTEM_PROMPT),
        ("user", USER_PROMPT),
    ],
)

chain = LLMChain(llm=llm, prompt=PROMPT)


preds = []
for idx, row in data[:].iterrows():
    text = row['geometry_subject'] + ' ' + row['predicate']
    text = row['geometry_subject'] + ' is within' if row['predicate']=='within' else text
    query = 'Retrieve a ' + row['geom_type'] + ' which ' + text


    pred = chain.invoke({'query':query})

    preds.append({'output': pred['text'], 'idx': idx, 'text': pred['text']})
    # print('record %d'%idx, 'predict: %s, actual: %s'%(pred['text'], (row['geom_type_subject'], row['predicate'], row['geom_type'])))
    if (idx+1)%50 == 0:
        res_llm = pd.DataFrame(preds)
        res_llm.to_csv(path+'result/temp/geom_task2_object_%s.csv'%(dir), index=False)
        print('----------------')
        print('record %d'%(idx+1), 'predict: %s, actual: %s'%(pred['text'], (row['geom_type_subject'], row['predicate'], row['geom_type'])))
        print('finish saving first %d predictions of %s, using %s.'%(idx+1, qe_mode, model_abbr))
        time.sleep(5)
res_llm = pd.DataFrame(preds)
print(res_llm.shape)
res_llm.to_csv(path+'result/temp/geom_task2_object_%s.csv'%(dir), index=False)

----------------
record 50 predict: POINT(-89.3055 43.1250), actual: ('LineString', 'contains', 'Point')
finish saving first 50 predictions of q2d_zs, using gpt4.
----------------
record 100 predict: LINESTRING (-89.389 43.074, -89.387 43.073), actual: ('Polygon', 'contains', 'LineString')
finish saving first 100 predictions of q2d_zs, using gpt4.
----------------
record 150 predict: LINESTRING (-89.5100 43.0950, -89.5000 43.0930, -89.4900 43.0900), actual: ('Polygon', 'contains', 'LineString')
finish saving first 150 predictions of q2d_zs, using gpt4.
----------------
record 200 predict: POLYGON ((-89.297 43.108, -89.2965 43.1085, -89.2961 43.109, -89.2972 43.1095, -89.297 43.108)), actual: ('Polygon', 'contains', 'Polygon')
finish saving first 200 predictions of q2d_zs, using gpt4.
----------------
record 250 predict: LINESTRING (-89.3636 43.0936, -88 43), actual: ('Point', 'touches', 'LineString')
finish saving first 250 predictions of q2d_zs, using gpt4.
----------------
record 300

In [ ]:
res_llm = pd.DataFrame(preds)
print(res_llm.shape)
res_llm.to_csv(path+'result/task2/geom_task2_object_%s.csv'%(dir), index=False)

(1040, 3)


### Retrieval

In [ ]:
qe_mode = 'q2d_zs'
dir = 'orig'

res_llm = pd.read_csv(path+'/result/task2/geom_task2_object_%s.csv'%( dir))
preds = res_llm.to_dict('record')

# res_llm = pd.read_csv(path+'/result/task2/geom_0_task2_%s.csv'%( qe_mode))
# preds0 = res_llm.to_dict('record')

# res_llm = pd.read_csv(path+'/result/task2/geom_1_task2_%s.csv'%( qe_mode))
# preds1 = res_llm.to_dict('record')

In [ ]:
import time
# llm_abbr = 'none'
# qe_mode = 'none'
repeat = 1

k_val = [5, 10, 20]

ranks = []
for idx, row in data.iloc[:].iterrows():
    text = row['geometry_subject'] + ' ' + row['predicate']
    text = row['geometry_subject'] + ' is ' + row['predicate'] if row['predicate']=='within' else text
    question = 'Retrieve a ' + row['geom_type'] + ' which ' + text + '. For example, ' + preds[idx]['text'] +'.'


    subjects = db.similarity_search_with_score(question, k=objects.shape[0])
    dists = []
    for subject in subjects:
        dists.append({'objectid': subject[0].metadata['objectid'], 'dist': subject[1]})
    dists = pd.DataFrame(dists)
    positive = set(intersections['objectid'][(intersections['subjectid']==row['subjectid'])&
                                            (intersections['predicate']==row['predicate'])&
                                             (intersections['geom_type']==row['geom_type'])]) - set([row['objectid']])

    dists = dists[~(dists['objectid'].isin(positive))].sort_values(by='dist')
    dists['rank'] = np.arange(dists.shape[0])+1
    rank = dists[dists['objectid']==row['objectid']].iloc[0]
    ranks.append({'idx': idx, 'rank': rank['rank'], 'dist': rank['dist'],
                  'list': dists['objectid'].iloc[:10].values.tolist()})

    if (idx+1)%50 == 0:
        res_search = pd.DataFrame(ranks)
        mrr = (1./res_search['rank'].values).mean()
        hits = []
        for k in k_val:
            hits.append(np.array(res_search['rank'].values<=k).mean())

        res_search.to_csv(path+'result/temp/task2_object_%s_%s.csv'%(qe_mode, dir), index=False)
        print('----------------')
        print('record %d'%(idx+1), 'mrr: %s, hits: %s'%(mrr, hits))
        print('finish saving first %d searching with %s QE, using %s as embedding model, %s as llm.with geom_type, % s'%(
            idx+1, qe_mode, embed_abbr, llm_abbr, include_type))
        time.sleep(5)

----------------
record 100 mrr: 0.4909577883528265, hits: [0.59, 0.81, 0.88]
finish saving first 100 searching with q2d_zs QE, using large as embedding model, gpt4 as llm.with geom_type, t


----------------
record 150 mrr: 0.5097650874707008, hits: [0.6, 0.7866666666666666, 0.8733333333333333]
finish saving first 150 searching with q2d_zs QE, using large as embedding model, gpt4 as llm.with geom_type, t


----------------
record 200 mrr: 0.5601065267051104, hits: [0.66, 0.815, 0.88]
finish saving first 200 searching with q2d_zs QE, using large as embedding model, gpt4 as llm.with geom_type, t


----------------
record 250 mrr: 0.47887068037074826, hits: [0.572, 0.708, 0.776]
finish saving first 250 searching with q2d_zs QE, using large as embedding model, gpt4 as llm.with geom_type, t


----------------
record 300 mrr: 0.44204759221377743, hits: [0.53, 0.6466666666666666, 0.7066666666666667]
finish saving first 300 searching with q2d_zs QE, using large as embedding model, gpt4 as llm.with geom_type, t


...

----------------
record 850 mrr: 0.33291268082309294, hits: [0.43411764705882355, 0.5152941176470588, 0.5670588235294117]
finish saving first 850 searching with q2d_zs QE, using large as embedding model, gpt4 as llm.with geom_type, t


----------------
record 900 mrr: 0.3218754783456081, hits: [0.4177777777777778, 0.5, 0.5522222222222222]
finish saving first 900 searching with q2d_zs QE, using large as embedding model, gpt4 as llm.with geom_type, t


----------------
record 950 mrr: 0.327882400152216, hits: [0.4263157894736842, 0.5094736842105263, 0.56]
finish saving first 950 searching with q2d_zs QE, using large as embedding model, gpt4 as llm.with geom_type, t


----------------
record 1000 mrr: 0.3423636664285614, hits: [0.444, 0.528, 0.578]
finish saving first 1000 searching with q2d_zs QE, using large as embedding model, gpt4 as llm.with geom_type, t


KeyboardInterrupt: 

In [ ]:
res_search = pd.DataFrame(ranks)
print(res_search.shape)
res_search.to_csv(path+'result/task2/task2_res_%s_%s.csv'%(qe_mode, repeat))

(1040, 4)


In [ ]:
k_val = [5, 10, 20]
res = pd.DataFrame(ranks)
mrr = (1./res['rank']).mean()
hits = []
for k in k_val:
    hits.append(np.array(res['rank'].values<=k).mean())

mrr, hits

(0.1689945870115535,
 [0.2326923076923077, 0.2798076923076923, 0.32019230769230766])

## Zero-shot-reverse

#### Generate pseudo geometry

In [ ]:
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.chains import LLMChain
import langchain
import time

# llm_abbr = 'gpt3_5'

# include_type = 'f'
k_val = [5, 10, 20]

qe_mode = 'q2d_zs'
dir = 'rev'

SYSTEM_PROMPT = """You will receive a spatial query containing the spatial relationship PREDICATE with the reference object B (WKT(B)), and
the desired geometry type of the subject to be retrieved.
Your task is to generate a plausible geometry A in WKT format that could be the result of the query.
The PREDICATE is defined according to the DE-9IM and Open Geospatial Consortium standards.

- Provide only the WKT(A) as the output, and nothing else.
- MAKE SURE the WKT(A) represents a valid Point, LineString, or Polygon as required by the query.
- MAKE SURE the PREDICATE satisfies the the dimension requirements defined by DE-9IM and OGC given Geometry Type A and Geometry Type B.
"""

USER_PROMPT = """
{query}.
"""


PROMPT = ChatPromptTemplate.from_messages(
    [
        ("system", SYSTEM_PROMPT),
        ("user", USER_PROMPT),
    ],
)

chain = LLMChain(llm=llm, prompt=PROMPT)


preds = []
for idx, row in data[:].iterrows():
    text = row['rev'] + ' ' + row['geometry_subject']
    # text =  ' is ' + text if row['rev']=='within' else text
    question = 'Retrieve a ' + row['geom_type'] +  ' which ' + text


    pred = chain.invoke({'query':question})

    preds.append({'output': pred['text'], 'idx': idx, 'text': pred['text']})
    # print('record %d'%idx, 'predict: %s, actual: %s'%(pred['text'], (row['geom_type_subject'], row['predicate'], row['geom_type'])))
    if (idx+1)%50 == 0:
        res_llm = pd.DataFrame(preds)
        res_llm.to_csv(path+'result/temp/geom_task2_object_%s.csv'%(dir), index=False)
        print('----------------')
        print('record %d'%(idx+1), 'predict: %s, actual: %s'%(pred['text'], (row['geom_type_subject'], row['predicate'], row['geom_type'])))
        print('finish saving first %d predictions of %s, using %s.'%(idx+1, qe_mode, model_abbr))
        time.sleep(5)
res_llm = pd.DataFrame(preds)
print(res_llm.shape)
res_llm.to_csv(path+'result/task2/geom_task2_object_%s.csv'%(dir), index=False)

----------------
record 50 predict: POINT(-89.3055 43.1250), actual: ('LineString', 'contains', 'Point')
finish saving first 50 predictions of q2d_zs, using gpt4.
----------------
record 100 predict: LINESTRING (-89.3900 43.0740, -89.3850 43.0750), actual: ('Polygon', 'contains', 'LineString')
finish saving first 100 predictions of q2d_zs, using gpt4.
----------------
record 150 predict: LINESTRING (-89.5100 43.0950, -89.5000 43.0900), actual: ('Polygon', 'contains', 'LineString')
finish saving first 150 predictions of q2d_zs, using gpt4.
----------------
record 200 predict: POLYGON ((-89.297 43.107, -89.2965 43.107, -89.2965 43.108, -89.297 43.108, -89.297 43.107)), actual: ('Polygon', 'contains', 'Polygon')
finish saving first 200 predictions of q2d_zs, using gpt4.
----------------
record 250 predict: LINESTRING(-89.3636 43.0936, -89.3635 43.0935), actual: ('Point', 'touches', 'LineString')
finish saving first 250 predictions of q2d_zs, using gpt4.
----------------
record 300 predict

In [ ]:
pred

{'query': 'Retrieve a Polygon which POLYGON ((-89.2972 43.0015, -89.2973 42.9978, -89.3022 42.9978, -89.3021 43.0014, -89.2989 43.001, -89.2981 43.0014, -89.2979 43.0015, -89.2972 43.0015)) equals. For example, POLYGON ((-89.2972 43.0015, -89.2973 42.9978, -89.3022 42.9978, -89.3021 43.0014, -89.2989 43.001, -89.2981 43.0014, -89.2979 43.0015, -89.2972 43.0015))',
 'text': 'POLYGON ((-89.2972 43.0015, -89.2973 42.9978, -89.3022 42.9978, -89.3021 43.0014, -89.2989 43.001, -89.2981 43.0014, -89.2979 43.0015, -89.2972 43.0015))'}

In [ ]:
start_idx = len(preds)
start_idx

40

In [ ]:
res_llm = pd.DataFrame(preds)
print(res_llm.shape)
res_llm.to_csv(path+'result/task2/geom_task2_object_%s.csv'%(dir), index=False)

(1040, 3)


### Retrieval

In [ ]:
qe_mode = 'q2d_zs'
dir = 'rev'

res_llm = pd.read_csv(path+'/result/task2/geom_task2_object_%s.csv'%( dir))
preds = res_llm.to_dict('record')

# res_llm = pd.read_csv(path+'/result/task2/geom_0_task2_%s.csv'%( qe_mode))
# preds0 = res_llm.to_dict('record')

# res_llm = pd.read_csv(path+'/result/task2/geom_1_task2_%s.csv'%( qe_mode))
# preds1 = res_llm.to_dict('record')

In [ ]:
import time
# llm_abbr = 'none'
# qe_mode = 'none'
repeat = 1

k_val = [5, 10, 20]

ranks = []
for idx, row in data.iloc[:440].iterrows():
    text = row['rev'] + ' ' + row['geometry_subject']
    question = 'Retrieve a ' + row['geom_type'] + ' which ' + text + '. For example, ' + preds[idx]['text'] +'.'


    subjects = db.similarity_search_with_score(question, k=objects.shape[0])
    dists = []
    for subject in subjects:
        dists.append({'objectid': subject[0].metadata['objectid'], 'dist': subject[1]})
    dists = pd.DataFrame(dists)
    positive = set(intersections['objectid'][(intersections['subjectid']==row['subjectid'])&
                                            (intersections['predicate']==row['predicate'])&
                                             (intersections['geom_type']==row['geom_type'])]) - set([row['objectid']])

    dists = dists[~(dists['objectid'].isin(positive))].sort_values(by='dist')
    dists['rank'] = np.arange(dists.shape[0])+1
    rank = dists[dists['objectid']==row['objectid']].iloc[0]
    ranks.append({'idx': idx, 'rank': rank['rank'], 'dist': rank['dist'],
                  'list': dists['objectid'].iloc[:10].values.tolist()})

    if (idx+1)%50 == 0:
        res_search = pd.DataFrame(ranks)
        mrr = (1./res_search['rank'].values).mean()
        hits = []
        for k in k_val:
            hits.append(np.array(res_search['rank'].values<=k).mean())

        res_search.to_csv(path+'result/temp/task2_object_%s_%s.csv'%(qe_mode, dir), index=False)
        print('----------------')
        print('record %d'%(idx+1), 'mrr: %s, hits: %s'%(mrr, hits))
        print('finish saving first %d searching with %s QE, using %s as embedding model, %s as llm.with geom_type, % s'%(
            idx+1, qe_mode, embed_abbr, llm_abbr, include_type))
        time.sleep(5)

----------------
record 50 mrr: 0.15004831627107446, hits: [0.3, 0.36, 0.42]
finish saving first 50 searching with q2d_zs QE, using large as embedding model, gpt4 as llm.with geom_type, t


----------------
record 100 mrr: 0.07625306829447469, hits: [0.15, 0.18, 0.23]
finish saving first 100 searching with q2d_zs QE, using large as embedding model, gpt4 as llm.with geom_type, t


----------------
record 150 mrr: 0.05841178701453981, hits: [0.11333333333333333, 0.13333333333333333, 0.16666666666666666]
finish saving first 150 searching with q2d_zs QE, using large as embedding model, gpt4 as llm.with geom_type, t


----------------
record 200 mrr: 0.054869036835617335, hits: [0.105, 0.12, 0.15]
finish saving first 200 searching with q2d_zs QE, using large as embedding model, gpt4 as llm.with geom_type, t


----------------
record 250 mrr: 0.08351738049398073, hits: [0.136, 0.184, 0.216]
finish saving first 250 searching with q2d_zs QE, using large as embedding model, gpt4 as llm.with geom_type, t


----------------
record 300 mrr: 0.0856942837611151, hits: [0.14333333333333334, 0.18666666666666668, 0.21666666666666667]
finish saving first 300 searching with q2d_zs QE, using large as embedding model, gpt4 as llm.with geom_type, t


----------------
record 350 mrr: 0.08801365970896816, hits: [0.14, 0.18571428571428572, 0.22285714285714286]
finish saving first 350 searching with q2d_zs QE, using large as embedding model, gpt4 as llm.with geom_type, t


----------------
record 400 mrr: 0.10491457348391606, hits: [0.1575, 0.2075, 0.2425]
finish saving first 400 searching with q2d_zs QE, using large as embedding model, gpt4 as llm.with geom_type, t


In [ ]:
start_idx = len(ranks)
start_idx

440

In [ ]:
print(qe_mode, dir)

q2d_zs rev


In [ ]:
ranks = ranks + ranks1

In [ ]:
res_search = pd.DataFrame(ranks)
print(res_search.shape)
res_search.to_csv(path+'result/task2/task2_object_%s_%s.csv'%(qe_mode, dir))

(1040, 4)


In [ ]:
k_val = [5, 10, 20]
res = pd.DataFrame(ranks)
mrr = (1./res['rank']).mean()
hits = []
for k in k_val:
    hits.append(np.array(res['rank'].values<=k).mean())

mrr, hits

(0.17886988657764435,
 [0.24807692307692308, 0.29423076923076924, 0.3326923076923077])

In [ ]:
(len(res))

600

In [ ]:
ranks1 = ranks[:]